## Aitt
* AI 추천 시간표
* 본인 학교/학과에 맞는 졸업적부 여부 및 졸업을 위해 필요한 수강과목등을 알려줌
* 대학교 중고마켓: 차별점으로는 수업이 다른 등의 많은 이유로 직접 대면하여 물건 거래가 어려울 때를 대비해 하나의 공유장소(대학교)를 사용하고 있음에 착안해 학교 내에 Aitt box를 설치해 물품을 보관하고 찾을 수 있도록 / Aitt box는 무인 택배함에서 착안한 아이디어로, 많은 짐에 비해 턱없이 부족한 사물함의 역할도 기대할 수 있음. 혹은 근처에 사는 자취/기숙사생을 통한 중개 거래 시스템에 대해서도 추후 검토할 예정

In [2]:
# 필요한 모듈 및 라이브러리를 임포트
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
import getpass

### class: HeadLessChrome
* 테스트 시에는 'headless'와 '--start fullscreen'옵션만 DataParsingFromPage 클래스에서 따로 선언하여 사용

In [3]:
class HeadLessChrome:
    def __init__(self):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('headless')
        self.options.add_argument('--start fullscreen')
        self.options.add_argument('window-size = 1920x1080')
        self.options.add_argument('disable--gpu')

### class: DataParsingFromPage
* 테스트 시에는 세종대학교 학사정보시스템에서만 구현(uis.sejong.ac.kr)

#### 성적조회페이지에 접근하는 방식에 대한 고민
* 현재는 xpath로 이동하며 성적조회페이지에 접근: 일반성이 떨어지는 문제
    
#### 성적 데이터를 파싱해올때의 문제점
* 성적 데이터가 iframe내에 있음
* xpath를 이용한 파싱: 화면에 표시되는 데이터만 파싱, 화면에 표시되지 않고 스크롤 아래 있으면 파생해오지 못함
* 전체화면을 스크린샷 찍어 사진에서 필요한 데이터를 추출: 스크롤이 필요한 iframe을 전체 스크린샷하는 방법

In [2]:
# 페이지 xpath를 이용해 "기이수성적조회" 페이지에 접근하는 클래스
class DataParsingFromPage():
    def __init__(self):
        # HeadLessChrome
        self.options = Options()
        self.options.add_argument('headless')
        self.options.add_argument('--start-maximized')
        self.driver = webdriver.Chrome('C:\chromedriver\chromedriver.exe', chrome_options=self.options)

    def access_from_url(self, url):
        self.driver.get(url)
        self.driver.implicitly_wait(1)
        
    def login(self, ID, PWD):
        # 사용자로부터 아이디와 별표로 표시되는 패스워드를 입력받음
        ID  = input('ID:')
        PWD = getpass.getpass('PWD:')      
        # 페이지에 아이디와 패스워드를 넘겨주어 로그인
        self.send_keys_by_name('id', ID)
        self.send_keys_by_name('password', PWD)
        self.click_xpath_element('//*[@id="logbtn"]')
        self.driver.implicitly_wait(1)

    def move_to_riaframe(self):
        # content frame으로 이동
        self.switch_frame_to('//*[@id="contentFrm"]')
        # 성적 조회 frame으로 이동
        self.switch_frame_to('//*[@id="riaframe"]')

    # 기이수성적조회페이지로 이동
    def move_to_check_page(self):
        # menu frame으로 이동
        self.switch_frame_to('//*[@id="menuFrm"]')
        # 수업/성적 클릭
        self.click_xpath_element('//*[@id="mainTable"]/tbody/tr[1]/td/table/tbody/tr/td/table/tbody/tr[3]/td')
        # 성적 및 강의평가 클릭
        self.click_xpath_element('//*[@id="SELF_STUDSELF_SUB_30"]/table/tbody/tr[5]/td')
        # 기이수성적조회 클릭
        self.click_xpath_element('//*[@id="SELF_STUDSELF_SUB_30SCH_SUG05_STUDSugRecordQ"]')
        # 부모 frame으로 이동
        self.switch_frame_to(None, is_parent=True)
        # 성적 조회 frame으로 이동
        self.move_to_riaframe()
        # 성적 조회 테이블로 이동
        self.driver.find_element_by_class_name('GLMainTable')

    def click_xpath_element(self, xpath):
        self.driver.find_element_by_xpath(xpath).click()
        sleep(1)

    def switch_frame_to(self, xpath, is_parent=None):
        if is_parent:
            self.driver.switch_to.parent_frame()
            return
        self.driver.switch_to.frame(self.driver.find_element_by_xpath(xpath))

    # 테스트에서는 저장된 아이디와 비밀번호를 이용하여 로그인
    def test_login_with_already_input(self):
        self.test_send_keys_by_name('id', '17011694')
        self.test_send_keys_by_name('password', 'please input password')
        self.click_xpath_element('//*[@id="logbtn"]')
        self.driver.implicitly_wait(0.5)